In [1]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [49]:
# Import and read the charity_data.csv.

import pandas as pd 
motor_df = pd.read_csv('Resources/Motor_Vehicle_Crashes_-_Vehicle_Information__Three_Year_Window.csv')
motor_df.head()

,Year,Case Vehicle ID,Vehicle Body Type,Registration Class,Action Prior to Accident,Type / Axles of Truck or Bus,Direction of Travel,Fuel Type,Vehicle Year,State of Registration,Number of Occupants,Engine Cylinders,Vehicle Make,Contributing Factor 1,Contributing Factor 1 Description,Contributing Factor 2,Contributing Factor 2 Description,Event Type,Partial VIN
0,2018,15232120,VAN TRUCK,AGRICULTURAL TRUCK,Parked,Not Entered,North,Gas,2015.0,NY,0.0,4.0,NISSA,HUMAN,Not Entered,HUMAN,Not Entered,Not Entered,3N6CM0KN2FK713460
1,2018,15232535,SUBURBAN,PASSENGER OR SUBURBAN,Making U Turn,Not Entered,South,Gas,2015.0,NY,1.0,6.0,HONDA,HUMAN,Not Entered,HUMAN,Unknown,Not Entered,5J6TF2H55FL001556
2,2018,15232536,SEDAN,Not Entered,Going Straight Ahead,Not Entered,North,Not Entered,NaN,PA,1.0,NaN,NaN,HUMAN,Not Entered,HUMAN,Unknown,Not Entered,NaN
3,2018,15232557,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Parked,Not Entered,South,Gas,2013.0,NY,5.0,4.0,NISSA,HUMAN,Passenger Distraction,VEHICLE,Other*,Not Entered,1N4AL3AP8DC229156
4,2018,15232556,SUBURBAN,PASSENGER OR SUBURBAN,Going Straight Ahead,Not Entered,South,Gas,2009.0,NY,2.0,8.0,CADIL,HUMAN,Not Applicable,HUMAN,Not Applicable,Not Entered,3GYFK22209G213337


In [50]:
# Drop all NA from rows

motor_df = motor_df.dropna()

In [51]:
# Get Names for indexs for which Contributing Factor  1 Description 1 is Not Entered
# Drop these rows

not_entered = motor_df[motor_df['Contributing Factor 1 Description'] == 'Not Entered'].index
motor_df.drop(not_entered, inplace = True)

In [52]:
# Get Names for indexs for which Contributing Factor  1 Description 1 is Not applicable
# Drop these rows

not_app = motor_df[motor_df['Contributing Factor 1 Description'] == 'Not Applicable'].index
motor_df.drop(not_app, inplace = True)

In [53]:
# Get Names for index for which Contributing Factor 1 Description is Unknown
# Drop these rows

unknown = motor_df[motor_df['Contributing Factor 1 Description'] == 'Unknown'].index
motor_df.drop(unknown, inplace = True)

In [54]:
# Get unknown values in direction of travel
# Drop these rows

unknown2 = motor_df[motor_df['Direction of Travel'] == 'Unknown'].index
motor_df.drop(unknown2, inplace = True)

In [55]:
# Get Not Applicable in Direction of Travel and drop these rows

not_app = motor_df[motor_df['Direction of Travel'] == 'Not Applicable'].index
motor_df.drop(not_app, inplace = True)

In [56]:
# Looking at the Direction of Travel column
directions = motor_df['Direction of Travel'].value_counts()
directions

# Placing the Direction of Travel into Bins
replace_direction = list(directions[directions < 12000].index)

# Replace in dataframe
for direction in replace_direction:
    motor_df["Direction of Travel"] = motor_df["Direction of Travel"].replace(direction, "Other")

In [57]:
# Looking at the Vehicle Make column
motor_count = motor_df['Vehicle Make'].value_counts()

# Determine which values to replace if counts are less than 5000
replace_Vehicle = list(motor_count[motor_count < 3000].index)
replace_Vehicle

# Replace in dataframe
for car in replace_Vehicle:
    motor_df["Vehicle Make"] = motor_df["Vehicle Make"].replace(car,"Other")
    
# Check to make sure binning was successful
motor_df['Vehicle Make'].value_counts()

TOYOT    57239
HONDA    46095
CHEVR    44133
FORD     44126
NISSA    35334
Other    33098
JEEP     21089
HYUND    18128
DODGE    16350
SUBAR    13317
KIA       9343
GMC       8263
BMW       8175
ME/BE     7529
VOLKS     7428
CHRYS     7369
LEXUS     5752
ACURA     5745
MAZDA     5685
RAM       4640
BUICK     4562
INFIN     4461
MITSU     3586
AUDI      3495
CADIL     3133
Name: Vehicle Make, dtype: int64

In [58]:
# Look at Regristration Class value counts for binning
class_count = motor_df['Registration Class'].value_counts()


# Determine which values to replace if counts are less than 4000
replace_class = list(class_count[class_count < 2000].index)

# Replace in dataframe
for Class in replace_class:
    motor_df['Registration Class'] = motor_df['Registration Class'].replace(Class,"Other")
    
# Check to make sure binning was successful
motor_df['Registration Class'].value_counts()

PASSENGER OR SUBURBAN         347712
OMNIBUS - TAXI                 20924
AGRICULTURAL TRUCK             14251
Other                          10599
COMMERCIAL                      9818
SPECIAL PASSENGER               5611
POLITICAL SUBDIVISION           4352
MOTORCYCLE                      2771
INTERNATIONAL REGISTRATION      2037
Name: Registration Class, dtype: int64

In [59]:
# Action Prior to accident
actions = motor_df['Action Prior to Accident'].value_counts()

# Determine which values to replace if counts are less than 4000
replace_action = list(actions[actions < 2000].index)

# Replace in dataframe
for action in replace_action:
    motor_df['Action Prior to Accident'] = motor_df['Action Prior to Accident'].replace(action,"Other")
    
# Check to make sure binning was successful
motor_df['Action Prior to Accident'].value_counts()

Going Straight Ahead        243029
Making Left Turn             46153
Backing                      23642
Slowing or Stopping          22816
Making Right Turn            21186
Changing Lanes               20355
Starting in Traffic           7221
Merging                       5984
Other                         5926
Starting from Parking         5644
Overtaking/Passing            4366
Making U Turn                 4236
Entering Parked Position      3154
Stopped in Traffic            2309
Parked                        2054
Name: Action Prior to Accident, dtype: int64

In [60]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME','Contributing Factor 1 Description'
# and 'Contributing Factor 2 Description'.

motor_df = motor_df.drop(['Case Vehicle ID', 'Partial VIN', 'Contributing Factor 2','Contributing Factor 2 Description'], axis=1)

In [61]:
motor_df['Year'].value_counts()

2019    317161
2018    100914
Name: Year, dtype: int64

In [62]:
# Recode Contributing Factor 1 to reflect integers instead of strings and define dependent variable

def Classification(X):
    if X == 2019:
        return 1
    else :
        return 0
    
motor_df['Year'] = motor_df['Year'].apply(Classification)
motor_df['Year'].value_counts()

1    317161
0    100914
Name: Year, dtype: int64

In [63]:
# Generate our categorical variable lists
motor_cat = motor_df.dtypes[motor_df.dtypes == "object"].index.tolist()
motor_cat

['Vehicle Body Type',
 'Registration Class',
 'Action Prior to Accident',
 'Type / Axles of Truck or Bus',
 'Direction of Travel',
 'Fuel Type',
 'State of Registration',
 'Vehicle Make',
 'Contributing Factor 1',
 'Contributing Factor 1 Description',
 'Event Type']

In [64]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(motor_df[motor_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(motor_cat)
encode_df.head()

,Vehicle Body Type_2 DOOR SEDAN,Vehicle Body Type_4 DOOR SEDAN,Vehicle Body Type_ALL TERRAIN VEHICLE,Vehicle Body Type_AMBULANCE,Vehicle Body Type_BUS (OMNIBUS),Vehicle Body Type_CEMENT MIXER,Vehicle Body Type_CONSTRUCTION EQUIPMENT,Vehicle Body Type_CONVERTIBLE,Vehicle Body Type_CUSTOM,Vehicle Body Type_DELIVERY TRUCK,...,"Event Type_Other*, Non-Collision","Event Type_Overturned, Non-Collision","Event Type_Pedestrian, Collision With","Event Type_Railroad Train, Collision With","Event Type_Ran Off Roadway Only, Non-Collision","Event Type_Sign Post, Collision With Fixed Object","Event Type_Snow Embankment, Collision With Fixed Object","Event Type_Submersion, Non-Collision","Event Type_Tree, Collision With Fixed Object",Event Type_Unknown
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
motor_df['Year'].value_counts()

1    317161
0    100914
Name: Year, dtype: int64

In [66]:
# Merge one-hot encoded features and drop the originals

motor_df = motor_df.merge(encode_df, left_index=True,right_index=True)
motor_df = motor_df.drop(motor_cat, 1)
motor_df.head()

,Year,Vehicle Year,Number of Occupants,Engine Cylinders,Vehicle Body Type_2 DOOR SEDAN,Vehicle Body Type_4 DOOR SEDAN,Vehicle Body Type_ALL TERRAIN VEHICLE,Vehicle Body Type_AMBULANCE,Vehicle Body Type_BUS (OMNIBUS),Vehicle Body Type_CEMENT MIXER,...,"Event Type_Other*, Non-Collision","Event Type_Overturned, Non-Collision","Event Type_Pedestrian, Collision With","Event Type_Railroad Train, Collision With","Event Type_Ran Off Roadway Only, Non-Collision","Event Type_Sign Post, Collision With Fixed Object","Event Type_Snow Embankment, Collision With Fixed Object","Event Type_Submersion, Non-Collision","Event Type_Tree, Collision With Fixed Object",Event Type_Unknown
3,0,2013.0,5.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0,2007.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0,2013.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0,2014.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,0,2006.0,3.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
motor_df.columns

In [68]:
# Split our preprocessed data into our features and target arrays
y = motor_df['Year'].values
X = motor_df.drop('Year', 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [69]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [70]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation  ="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation ="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation ="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 1728      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 1,779
Trainable params: 1,779
Non-trainable params: 0
_________________________________________________________________


In [71]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [72]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
1268/1268 [==============================] - 3s 2ms/step - loss: 2.3528 - accuracy: 0.9235
Epoch 2/100
1268/1268 [==============================] - 2s 1ms/step - loss: 3.1557e-04 - accuracy: 1.0000
Epoch 3/100
1268/1268 [==============================] - 2s 1ms/step - loss: 1.0436e-04 - accuracy: 1.0000
Epoch 4/100
1268/1268 [==============================] - 2s 1ms/step - loss: 4.4526e-05 - accuracy: 1.0000
Epoch 5/100
1268/1268 [==============================] - 2s 1ms/step - loss: 2.0966e-05 - accuracy: 1.0000
Epoch 6/100
1268/1268 [==============================] - 2s 1ms/step - loss: 1.0338e-05 - accuracy: 1.0000
Epoch 7/100
1268/1268 [==============================] - 2s 2ms/step - loss: 5.2110e-06 - accuracy: 1.0000
Epoch 8/100
1268/1268 [==============================] - 2s 1ms/step - loss: 2.6559e-06 - accuracy: 1.0000
Epoch 9/100
1268/1268 [==============================] - 2s 1ms/step - loss: 1.3628e-06 - accuracy: 1.0000
Epoch 10/100
1268/1268 [=================

Epoch 77/100
1268/1268 [==============================] - 2s 1ms/step - loss: 1.1720e-14 - accuracy: 1.0000
Epoch 78/100
1268/1268 [==============================] - 2s 1ms/step - loss: 1.1487e-14 - accuracy: 1.0000
Epoch 79/100
1268/1268 [==============================] - 2s 1ms/step - loss: 1.1259e-14 - accuracy: 1.0000
Epoch 80/100
1268/1268 [==============================] - 2s 1ms/step - loss: 1.1036e-14 - accuracy: 1.0000
Epoch 81/100
1268/1268 [==============================] - 2s 2ms/step - loss: 1.0817e-14 - accuracy: 1.0000
Epoch 82/100
1268/1268 [==============================] - 2s 1ms/step - loss: 1.0602e-14 - accuracy: 1.0000
Epoch 83/100
1268/1268 [==============================] - 2s 2ms/step - loss: 1.0391e-14 - accuracy: 1.0000
Epoch 84/100
1268/1268 [==============================] - 2s 2ms/step - loss: 1.0184e-14 - accuracy: 1.0000
Epoch 85/100
1268/1268 [==============================] - 2s 2ms/step - loss: 9.9819e-15 - accuracy: 1.0000
Epoch 86/100
1268/1268 [====

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")